<a href="https://colab.research.google.com/github/kenzeng24/python-projects/blob/main/sarcasm-detection/sarcasm_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ktrain -q

     |████████████████████████████████| 25.3 MB 92 kB/s 
     |████████████████████████████████| 6.8 MB 12.4 MB/s 
     |████████████████████████████████| 981 kB 55.8 MB/s 
     |████████████████████████████████| 263 kB 48.9 MB/s 
     |████████████████████████████████| 1.9 MB 56.0 MB/s 
     |████████████████████████████████| 1.2 MB 52.7 MB/s 
     |████████████████████████████████| 468 kB 54.5 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
     |████████████████████████████████| 895 kB 58.2 MB/s 


In [3]:
import pandas as pd
import numpy as np
import ktrain
import tensorflow as tf
import json 
from ktrain import text
from sklearn.model_selection import train_test_split

In [4]:
ls

sample_data/


In [5]:
#import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
url = "https://raw.githubusercontent.com/kenzeng24/python-projects/main/sarcasm-detection/data/Sarcasm_Headlines_Dataset_v2.json"
raw = pd.read_json(url, lines = True)

In [6]:
data = raw[['headline', 'is_sarcastic']]

#shuffle data to split into train, test sets
data_train, data_test = train_test_split(
    data, train_size = 0.8, random_state = 42, shuffle = True)
data_train.head() 

,headline,is_sarcastic
4724,"how trump really feels about queer people, exp...",0
26559,egyptian death sentence for soccer fans puts p...,0
24613,stripper failing school she's working self thr...,1
9048,texas attorney general ken paxton indicted,0
69,congresswoman fights for gun control because s...,0


In [7]:
(train, val, preproc) = text.texts_from_df(
    train_df=data_train, 
    text_column='headline', 
    label_columns='is_sarcastic',
    val_df = data_test,
    maxlen = 400,
    preprocess_mode = 'distilbert')

['not_is_sarcastic', 'is_sarcastic']
       not_is_sarcastic  is_sarcastic
4724                1.0           0.0
26559               1.0           0.0
24613               0.0           1.0
9048                1.0           0.0
69                  1.0           0.0
['not_is_sarcastic', 'is_sarcastic']
       not_is_sarcastic  is_sarcastic
27927               1.0           0.0
1660                0.0           1.0
96                  0.0           1.0
6237                0.0           1.0
6650                1.0           0.0



preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


In [8]:
# initialize a distilbert model and learner
model = text.text_classifier(
    name = 'distilbert', 
    train_data = train, 
    preproc=preproc)

learner = ktrain.get_learner(
    model = model,
    train_data = train,
    val_data = val,
    batch_size = 6)



Is Multi-Label? False
maxlen is 400



done.


In [9]:
learner.fit_onecycle(lr = 2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
3816/3816 [==============================] - 1110s 287ms/step - loss: 0.3148 - accuracy: 0.8634 - val_loss: 0.2460 - val_accuracy: 0.8959
Epoch 2/2
3816/3816 [==============================] - 1101s 287ms/step - loss: 0.1358 - accuracy: 0.9496 - val_loss: 0.1842 - val_accuracy: 0.9299


In [10]:
#from google.colab import drive
# drive.mount('/content/drive')
predictor = ktrain.get_predictor(learner.model, preproc)
# predictor.save('/content/drive/My Drive/distilbert')

Mounted at /content/drive


In [11]:
# sarcrastic 
predictor.predict(["stripper failing school"], return_proba=True)

array([[0.01293929, 0.9870608 ]], dtype=float32)

In [12]:
# not sarcastic 
predictor.predict(["texas govenor fights for gun control bill"], return_proba=True)

array([[0.9959384 , 0.00406159]], dtype=float32)

In [19]:
# sarcastic headlines 
headlines = ["Donald Trump reduces America's unemployment rate to 0 by killing all Americans",
             "Man sentenced to death for being unkillable",
             "Black man fined two million dollars and faces ten years in prison after stealing two dollars", 
             "Ted Cruz aims to reduce Texas' carbon emission by negative twenty percent", 
             "Aliens are convinced that there's no intelligence on Earth after seeing a Republican rally"] 

outputs = predictor.predict(headlines, return_proba=True)

for i, (_, sarc_score) in enumerate(outputs): 
    print("headline: {}".format(headlines[i]))
    print("score: {}".format(sarc_score))
    sentiment = "not sarcastic"
    if sarc_score > 0.5: 
        sentiment = "sarcastic"
    print("output: this is {}".format(sentiment))
    print()

headline: Donald Trump reduces America's unemployment rate to 0 by killing all Americans
score: 0.0938887745141983
output: this is not sarcastic

headline: Man sentenced to death for being unkillable
score: 0.18744656443595886
output: this is not sarcastic

headline: Black man fined two million dollars and faces ten years in prison after stealing two dollars
score: 0.06763703376054764
output: this is not sarcastic

headline: Ted Cruz aims to reduce Texas' carbon emission by negative twenty percent
score: 0.01029697060585022
output: this is not sarcastic

headline: Aliens are convinced that there's no intelligence on Earth after seeing a Republican rally
score: 0.013377552852034569
output: this is not sarcastic



In [18]:
headlines = ["Donald Trump makes a peace deal between Palestine and Israel", 
             "President Trump faces scrutiny from oppositin after claiming that COVID19 is fake", 
             "Content creators are resorting to soft-core porn to make money", 
             "Children are spending the majority of their time online"] 

outputs = predictor.predict(headlines, return_proba=True)

for i, (_, sarc_score) in enumerate(outputs): 
    print("headline: {}".format(headlines[i]))
    print("score: {}".format(sarc_score))
    sentiment = "not sarcastic"
    if sarc_score > 0.5: 
        sentiment = "sarcastic"
    print("output: this is {}".format(sentiment))
    print()

headline: Donald Trump makes a peace deal between Palestine and Israel
score: 0.0010757619747892022
output: this is not sarcastic

headline: President Trump faces scrutiny from oppositin after claiming that COVID19 is fake
score: 0.0026526760775595903
output: this is not sarcastic

headline: Content creators are resorting to soft-core porn to make money
score: 0.005389266647398472
output: this is not sarcastic

headline: Children are spending the majority of their time online
score: 0.002230547135695815
output: this is not sarcastic

